#Book Recommendation

Dataset yang digunakan dalam proyek ini adalah Book-Crossing Dataset. Dataset ini berisi informasi tentang pengguna, buku, dan rating yang diberikan oleh pengguna terhadap buku. Dataset ini dapat diunduh melalui tautan berikut:
[Book-Crossing Dataset](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?select=Ratings.csv) atau https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?select=Ratings.csv


**Variabel pada dataset**

| Dataset  | Variabel              | Deskripsi |
|:---------|:----------------------|:----------|
| Users    | User-ID                | ID unik untuk setiap pengguna |
|          | Location               | Lokasi pengguna|
|          | Age                    | Usia pengguna |
| Books    | ISBN                   | Nomor ISBN buku |
|          | Book-Title             | Judul buku. |
|          | Book-Author            | Nama penulis buku|
|          | Year-Of-Publication    | Tahun terbit buku. |
|          | Publisher              | Nama penerbit buku. |
|          | Image-URL-S            | URL gambar sampul berukuran kecil. |
|          | Image-URL-M            | URL gambar sampul berukuran sedang. |
|          | Image-URL-L            | URL gambar sampul berukuran besar. |
| Ratings  | User-ID                | ID pengguna yang memberikan rating. |
|          | ISBN                   | ISBN buku yang dinilai. |
|          | Book-Rating            | Rating yang diberikan (0 untuk implicit, 1-10 untuk explicit rating). |

##Data Understanding

Import library yang dibutuhkan

In [1]:
# !pip install --upgrade numpy
# !pip install --upgrade gensim
# !pip install scikit-surprise

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import shutil
import os
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, GridSearchCV

Mendownload dataset yang berasar dari platform kaggle

In [3]:
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")
print("Path to dataset files:", path)
print("Files in dataset folder:")
print(os.listdir(path))

destination_path = "/content"
for file_name in os.listdir(path):
    full_file_name = os.path.join(path, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, destination_path)

print("✅ Dataset berhasil disalin ke /content")
print("Isi folder /content:")
print(os.listdir("/content"))

Path to dataset files: /kaggle/input/book-recommendation-dataset
Files in dataset folder:
['Ratings.csv', 'Users.csv', 'classicRec.png', 'Books.csv', 'DeepRec.png', 'recsys_taxonomy2.png']
✅ Dataset berhasil disalin ke /content
Isi folder /content:
['.config', 'classicRec.png', 'Users.csv', 'Ratings.csv', 'Books.csv', 'recsys_taxonomy2.png', 'DeepRec.png', 'kaggle.json', 'sample_data']


Meload dataset-dataset yang telah didownload tadi kedalam sebuah variabel menggunakan library pandas

In [4]:
book = pd.read_csv('/content/Books.csv')
rating = pd.read_csv('/content/Ratings.csv')
user = pd.read_csv('/content/Users.csv')

<ipython-input-4-423f50df06f9>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('/content/Books.csv')


Memeriksa variabel-variabel pada dataset book, rating dan user

In [5]:
book.info()
rating.info()
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), obje

Dataset yang digunakan terdiri dari tiga bagian utama: data buku, data pengguna, dan data rating. Dataset buku mencakup 271.360 entri dengan 8 atribut, seperti ISBN, judul, penulis, tahun terbit, penerbit, dan URL gambar sampul, di mana sebagian besar data bersifat lengkap meskipun terdapat sedikit nilai null pada kolom penulis, penerbit, dan gambar berukuran besar. Dataset pengguna memuat 278.858 entri yang berisi informasi User-ID, lokasi, dan usia, namun hanya sekitar 60% data usia yang tersedia. Sementara itu, dataset rating berisi 1.149.780 entri, terdiri dari pasangan User-ID dan ISBN dengan nilai rating yang diberikan pengguna, tanpa adanya nilai null.

###Univariate Exploratory Data Analysis

**Berikut adalah daftar variabel pada dataset yang digunakan:**

**Dataset buku:**
- ISBN : merupakan kode unik untuk setiap buku.
- Book-Title : merupakan judul dari buku.
- Book-Author : merupakan nama penulis dari buku.
- Year-Of-Publication : merupakan tahun terbit dari buku.
- Publisher : merupakan nama penerbit dari buku.
- Image-URL-S : merupakan URL gambar sampul buku berukuran kecil.
- Image-URL-M : merupakan URL gambar sampul buku berukuran sedang.
- Image-URL-L : merupakan URL gambar sampul buku berukuran besar.

**Dataset Pengguna:**
- User-ID : merupakan identitas unik dari setiap pengguna.
- Location : merupakan lokasi pengguna berdasarkan kota, negara bagian, dan negara.
- Age : merupakan usia pengguna.

**Dataset Rating:**
- User-ID : merupakan identitas unik dari pengguna yang memberi rating.
- ISBN : merupakan kode unik dari buku yang diberi rating.
- Book-Rating : merupakan nilai rating yang diberikan pengguna terhadap buku.
--------------------------------------------------------------------------------

####Dataset Buku

In [6]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


**Variabel ISBN**

In [7]:
print('Jumlah ISBN unik:', book['ISBN'].nunique())
print('Jumlah missing ISBN:', book['ISBN'].isnull().sum())

Jumlah ISBN unik: 271360
Jumlah missing ISBN: 0


Merupakan kode unik yang digunakan untuk mengidentifikasi setiap buku secara individual. Dalam dataset ini, seluruh 271.360 baris memiliki nilai ISBN yang unik tanpa nilai kosong.

--------------------------------------------------------------------------------

**Variabel Book-Title**

In [8]:
print('Jumlah judul unik:', book['Book-Title'].nunique())
print('Judul paling umum:\n', book['Book-Title'].value_counts().head(5))

Jumlah judul unik: 242135
Judul paling umum:
 Book-Title
Selected Poems       27
Little Women         24
Wuthering Heights    21
The Secret Garden    20
Dracula              20
Name: count, dtype: int64


Menunjukkan judul buku. Dari 271.360 data, terdapat 242.135 judul unik, menunjukkan adanya beberapa buku yang memiliki judul yang sama (kemungkinan edisi berbeda atau re-publish). Judul yang paling sering muncul antara lain Selected Poems, Little Women, dan Wuthering Heights.

--------------------------------------------------------------------------------

**Variabel Book Author**

In [9]:
print('Jumlah penulis unik:', book['Book-Author'].nunique())
print('Jumlah missing:', book['Book-Author'].isnull().sum())
print('Penulis paling banyak muncul:\n', book['Book-Author'].value_counts().head(5))

Jumlah penulis unik: 102022
Jumlah missing: 2
Penulis paling banyak muncul:
 Book-Author
Agatha Christie        632
William Shakespeare    567
Stephen King           524
Ann M. Martin          423
Carolyn Keene          373
Name: count, dtype: int64


Menampilkan nama penulis dari setiap buku. Terdapat 102.022 penulis unik dan hanya 2 nilai yang hilang. Penulis yang paling sering muncul dalam dataset ini adalah Agatha Christie, William Shakespeare, dan Stephen King, yang masing-masing memiliki ratusan judul.

--------------------------------------------------------------------------------

**Variabel Year-Of-Publication**

In [10]:
book['Year-Of-Publication'] = pd.to_numeric(book['Year-Of-Publication'], errors='coerce')
print(book['Year-Of-Publication'].describe())
print('Tahun terbit unik:', book['Year-Of-Publication'].nunique())
print('Tahun terbit paling umum:\n', book['Year-Of-Publication'].value_counts().head())


count    271357.000000
mean       1959.760817
std         257.994226
min           0.000000
25%        1989.000000
50%        1995.000000
75%        2000.000000
max        2050.000000
Name: Year-Of-Publication, dtype: float64
Tahun terbit unik: 116
Tahun terbit paling umum:
 Year-Of-Publication
2002.0    17627
1999.0    17431
2001.0    17359
2000.0    17232
1998.0    15766
Name: count, dtype: int64


Merupakan tahun terbit buku yang awalnya bertipe objek dan telah dikonversi menjadi numerik. Rentangnya sangat luas (0 hingga 2050), menunjukkan adanya outlier atau data error. Tahun yang paling umum muncul adalah antara 1998 hingga 2002.

--------------------------------------------------------------------------------

**Variabel Publisher**

In [11]:
print('Jumlah penerbit unik:', book['Publisher'].nunique())
print('Jumlah missing:', book['Publisher'].isnull().sum())
print('Penerbit terbanyak:\n', book['Publisher'].value_counts().head())

Jumlah penerbit unik: 16807
Jumlah missing: 2
Penerbit terbanyak:
 Publisher
Harlequin           7535
Silhouette          4220
Pocket              3905
Ballantine Books    3783
Bantam Books        3646
Name: count, dtype: int64


Menunjukkan penerbit buku. Terdapat 16.807 penerbit unik dengan hanya 2 data yang hilang. Beberapa penerbit yang paling banyak muncul adalah Harlequin, Silhouette, dan Pocket.

--------------------------------------------------------------------------------

**Variabel Image URL**

In [12]:
print('Jumlah missing image kecil:', book['Image-URL-S'].isnull().sum())
print('Jumlah missing image sedang:', book['Image-URL-M'].isnull().sum())
print('Jumlah missing image besar:', book['Image-URL-L'].isnull().sum())

Jumlah missing image kecil: 0
Jumlah missing image sedang: 0
Jumlah missing image besar: 3


Ketiganya adalah URL gambar sampul buku dalam tiga ukuran berbeda. Kolom ini tidak memiliki nilai kosong, kecuali Image-URL-L yang memiliki 3 missing value. Data ini bersifat pelengkap dan lebih relevan untuk aplikasi berbasis tampilan visual.

--------------------------------------------------------------------------------

**Kesimpulan**

ISBN bersifat unik dan tidak memiliki nilai kosong, sedangkan Book-Title memiliki 242.135 judul unik dari total 271.360 data. Menunjukkan adanya banyak buku dengan judul sama, dengan Selected Poems menjadi yang paling sering muncul. Book-Author menunjukkan bahwa Agatha Christie adalah penulis yang paling sering muncul dengan 632 buku, diikuti oleh William Shakespeare dan Stephen King, sementara hanya terdapat dua nilai kosong. Year-Of-Publication memiliki sebaran yang luas dengan outlier seperti nilai 0 dan 2050, namun mayoritas buku diterbitkan antara tahun 1998 hingga 2002, dengan tahun 2002 sebagai yang paling umum. Publisher memiliki 16.807 penerbit unik, dengan Harlequin mendominasi sebanyak 7.535 buku, dan hanya dua data yang kosong. Tiga kolom gambar tidak memiliki missing value kecuali Image-URL-L, yang hanya memiliki tiga data kosong.

--------------------------------------------------------------------------------

####Dataset Pengguna

In [13]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


**Variabel User-ID**

In [14]:
print("Jumlah User unik:", user['User-ID'].nunique())
print("Apakah ada missing User-ID?:", user['User-ID'].isnull().sum())

Jumlah User unik: 278858
Apakah ada missing User-ID?: 0


Terdapat 278.858 nilai unik pada kolom User-ID, menandakan bahwa setiap pengguna dalam dataset bersifat unik. Tidak ditemukan nilai yang hilang pada kolom ini.

--------------------------------------------------------------------------------

**Variabel Location**

In [15]:
print("Jumlah lokasi unik:", user['Location'].nunique())
print(user['Location'].value_counts().head())
print("Apakah ada missing Location?:", user['Location'].isnull().sum())

Jumlah lokasi unik: 57339
Location
london, england, united kingdom       2506
toronto, ontario, canada              2250
sydney, new south wales, australia    1744
melbourne, victoria, australia        1708
portland, oregon, usa                 1629
Name: count, dtype: int64
Apakah ada missing Location?: 0


Terdapat 57.339 lokasi unik dalam kolom Location, yang menunjukkan keragaman geografis pengguna. Lokasi yang paling sering muncul adalah London, England, United Kingdom sebanyak 2.506 kali, diikuti oleh Toronto, Ontario, Canada dan Sydney, New South Wales, Australia. Tidak terdapat data yang hilang pada kolom ini.

--------------------------------------------------------------------------------

**Variabel Age**

In [16]:
print(user['Age'].describe())
print("Umur unik:", user['Age'].nunique())
print("Umur tertinggi:", user['Age'].max())
print("Umur terendah:", user['Age'].min())

count    168096.000000
mean         34.751434
std          14.428097
min           0.000000
25%          24.000000
50%          32.000000
75%          44.000000
max         244.000000
Name: Age, dtype: float64
Umur unik: 165
Umur tertinggi: 244.0
Umur terendah: 0.0


Dari total data, hanya 168.096 (sekitar 60%) yang memiliki nilai usia. Rata-rata usia pengguna adalah sekitar 35 tahun dengan simpangan baku 14,43 tahun. Terdapat nilai ekstrim seperti usia minimum 0 tahun dan maksimum 244 tahun yang kemungkinan merupakan kesalahan input atau outlier.

--------------------------------------------------------------------------------

**Kesimpulan**

Dataset pengguna terdiri dari 278.858 entri unik pada kolom User-ID tanpa adanya nilai yang hilang, menunjukkan representasi yang lengkap untuk setiap pengguna. Kolom Location mencerminkan keberagaman geografis yang tinggi dengan lebih dari 57 ribu lokasi berbeda, di mana kota-kota besar seperti London, Toronto, dan Sydney muncul paling sering. Sementara itu, variabel Age hanya terisi pada sekitar 60% data, dengan usia rata-rata sekitar 35 tahun, namun terdapat nilai-nilai ekstrem seperti 0 dan 244 tahun yang menunjukkan adanya kemungkinan outlier atau kesalahan input yang perlu ditangani pada tahap pembersihan data.

--------------------------------------------------------------------------------

####Dataset Rating

In [17]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


**Variabel User-ID**

In [18]:
print("Jumlah User unik:", rating['User-ID'].nunique())
print("Apakah ada missing User-ID?:", rating['User-ID'].isnull().sum())

Jumlah User unik: 105283
Apakah ada missing User-ID?: 0


Terdapat 105.283 pengguna unik dalam dataset ini, dan tidak ditemukan nilai yang hilang pada kolom User-ID.

--------------------------------------------------------------------------------

**Variabel ISBN**

In [19]:
print("Jumlah ISBN unik:", rating['ISBN'].nunique())
print("ISBN paling sering muncul:\n", rating['ISBN'].value_counts().head())
print("Apakah ada missing ISBN?:", rating['ISBN'].isnull().sum())

Jumlah ISBN unik: 340556
ISBN paling sering muncul:
 ISBN
0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
Name: count, dtype: int64
Apakah ada missing ISBN?: 0


Jumlah ISBN unik yang tercatat adalah 340.556. ISBN yang paling sering muncul adalah 0971880107 sebanyak 2.502 kali, diikuti oleh 0316666343 sebanyak 1.295 kali. Tidak ada nilai kosong pada kolom ini.

--------------------------------------------------------------------------------

**Variabel book rating**

In [20]:
print("Statistik deskriptif Book-Rating:\n", rating['Book-Rating'].describe())
print("Rating unik:", rating['Book-Rating'].nunique())
print("Distribusi rating:\n", rating['Book-Rating'].value_counts().sort_index())

Statistik deskriptif Book-Rating:
 count    1.149780e+06
mean     2.866950e+00
std      3.854184e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      7.000000e+00
max      1.000000e+01
Name: Book-Rating, dtype: float64
Rating unik: 11
Distribusi rating:
 Book-Rating
0     716109
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
Name: count, dtype: int64


Rating buku memiliki 1.149.780 entri dengan nilai rata-rata 2.87 dan standar deviasi sebesar 3.85. Nilai rating berkisar antara 0 hingga 10. Sebanyak 716.109 entri (sekitar 62%) memiliki rating 0, yang biasanya menandakan tidak ada rating yang diberikan. Rating 10 muncul sebanyak 78.610 kali, menunjukkan sejumlah pengguna memberikan skor maksimal. Distribusi lainnya menunjukkan peningkatan jumlah data seiring naiknya rating, terutama dari nilai 5 hingga 8.

--------------------------------------------------------------------------------

**Kesimpulan**

Dataset ini mencakup 105.283 pengguna unik dan 340.556 ISBN unik tanpa ada data yang hilang di kedua kolom tersebut. ISBN 0971880107 merupakan buku yang paling banyak dinilai. Distribusi rating menunjukkan bahwa sebagian besar pengguna tidak memberikan rating (nilai 0), sedangkan skor tinggi seperti 8, 9, dan 10 cukup banyak diberikan, mengindikasikan kecenderungan pengguna untuk memberi rating positif terhadap buku yang mereka baca.

--------------------------------------------------------------------------------

##Data Preparation

###Data Preprocessing

 📘 Tabel Ringkasan Preprocessing

| Dataset          | Variabel               | Masalah Ditemukan                                | Tindakan Preprocessing                                 |
|------------------|------------------------|--------------------------------------------------|--------------------------------------------------------|
| Buku             | Book-Title            | Terdapat judul buku yang sama                         | Hapus judul buku yang sama                        |
| Buku             | Book-Author            | Terdapat 2 missing value                         | Hapus baris dengan nilai kosong                        |
| Buku             | Publisher              | Terdapat 2 missing value                         | Hapus baris dengan nilai kosong                        |
| Buku             | Year-Of-Publication    | Nilai tidak valid: 0 dan > 2025                  | Menggantinya dengan nilai median |
| Buku             |  Image-URL | Variabel tidak dibutuhkan                  | Menghapus variabel (drop) |
| Pengguna         | Age                    | Nilai outlier: 0 dan > 100                       | Menggantinya dengan nilai median |

--------------------------------------------------------------------------------




Menghapus judul buku yang sama atau duplikat

In [21]:
book = book.drop_duplicates(subset=['Book-Title']).reset_index(drop=True)

Menghapus baris dengan Book-Author yang kosong

In [22]:
book = book[book['Book-Author'].notnull()]

Menghapus baris dengan Publisher yang kosong

In [23]:
book = book[book['Publisher'].notnull()]

Menghapus variabel Imgae-URL

In [24]:
book.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)

<ipython-input-24-b99cd9a0f20c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)


Memperbaiki kolom Year-Of-Publication

In [25]:
# Ganti nilai tahun yang tidak valid (0 atau >2025) dengan NaN
book.loc[(book['Year-Of-Publication'] == 0) | (book['Year-Of-Publication'] > 2025), 'Year-Of-Publication'] = np.nan

In [26]:
# Ganti NaN dengan median tahun
median_year = book['Year-Of-Publication'].median()
book['Year-Of-Publication'].fillna(median_year, inplace=True)

<ipython-input-26-dff23fa1f252>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  book['Year-Of-Publication'].fillna(median_year, inplace=True)
<ipython-input-26-dff23fa1f252>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book['Year-Of-Publication'].fillna(median_year, inplace=True)


Memperbaiki Kolom Age

Mengganti nilai yang tidak realistis (umur 0 atau >100) dengan NaN

In [27]:
user.loc[(user['Age'] < 5) | (user['Age'] > 100), 'Age'] = np.nan

Mengganti NaN dengan median usia

In [28]:
median_age = user['Age'].median()
user['Age'].fillna(median_age, inplace=True)

<ipython-input-28-f0b193ffa30b>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user['Age'].fillna(median_age, inplace=True)


**Hasil setelah Preprocessing**

In [29]:
print("Books dataset setelah preprocessing:")
print(book.info())
print("\nUsers dataset setelah preprocessing:")
print(user.info())

Books dataset setelah preprocessing:
<class 'pandas.core.frame.DataFrame'>
Index: 242132 entries, 0 to 242134
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 242132 non-null  object 
 1   Book-Title           242132 non-null  object 
 2   Book-Author          242132 non-null  object 
 3   Year-Of-Publication  242132 non-null  float64
 4   Publisher            242132 non-null  object 
dtypes: float64(1), object(4)
memory usage: 11.1+ MB
None

Users dataset setelah preprocessing:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       278858 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB
None


**Kesimpulan**

Setelah dilakukan preprocessing, data pada kolom Book-Author dan Publisher yang memiliki nilai kosong dihapus untuk memastikan kualitas data yang bersih dan valid. Nilai tidak logis pada Year-Of-Publication, seperti tahun 0 dan di atas 2025, disesuaikan dengan nilai median agar tetap merepresentasikan distribusi data yang wajar. Kolom Image-URL dihapus karena tidak relevan terhadap analisis atau pemodelan. Untuk data pengguna, nilai usia yang ekstrem seperti 0 dan di atas 100 dianggap sebagai outlier dan digantikan dengan nilai median guna menjaga keakuratan distribusi usia pengguna. Dengan langkah-langkah ini, dataset kini lebih bersih, konsisten, dan siap digunakan dalam tahap analisis lebih lanjut atau pembangunan model rekomendasi.

--------------------------------------------------------------------------------

Memeriksa Data Book untuk persiapan Modelling Content Based Filtering

In [30]:
book

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company
...,...,...,...,...,...
242130,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993.0,Ballantine Books
242131,0525447644,From One to One Hundred,Teri Sloat,1991.0,Dutton Books
242132,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004.0,HarperSanFrancisco
242133,0192126040,Republic (World's Classics),Plato,1996.0,Oxford University Press


Melakukan Filter pada dataset rating untuk persiapan Collaborative Filtering

In [31]:
filtered_ratings = rating[
    (rating['User-ID'].isin(rating['User-ID'].value_counts()[rating['User-ID'].value_counts() > 20].index)) &
    (rating['ISBN'].isin(rating['ISBN'].value_counts()[rating['ISBN'].value_counts() > 20].index))
]

In [32]:
filtered_ratings

,User-ID,ISBN,Book-Rating
33,276762,0380711524,5
34,276762,0451167317,0
173,276847,0446364193,0
175,276847,3379015180,0
182,276847,3426029553,8
...,...,...,...
1149737,276688,0688143687,0
1149738,276688,0688156134,8
1149743,276688,0836218655,10
1149744,276688,0836236688,10


##Model Development dengan Content Based Filtering dengan Word2Vec

Memisahkan variabel variabel yang akan dibutuhkan untuk model

In [33]:
text_data = book['Book-Title'] + ' ' + book['Book-Author'] + ' ' + book['Publisher']

Tokenize dan train Word2Vec

In [34]:
sentences = [text.split() for text in text_data]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1)


Fungsi untuk mencari kesamaan buku berdasarkan hasil embedding Word2Vec

In [35]:
def get_book_recommendations(title_input, top_n=10, return_similarities=False):
    title_vector = np.mean(
        [model.wv[word] for word in title_input.split() if word in model.wv], axis=0
    )

    similarities = []
    for idx, row in book.iterrows():
        book_title = row['Book-Title']
        book_vector = np.mean(
            [model.wv[word] for word in book_title.split() if word in model.wv], axis=0
        )
        if book_vector is not None and not np.isnan(book_vector).any():
            similarity = cosine_similarity([title_vector], [book_vector])[0][0]
            similarities.append((idx, similarity))

    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    recommended = similarities[:top_n]

    result = []
    for idx, score in recommended:
        result.append({
            'ISBN': book.iloc[idx]['ISBN'],
            'Title': book.iloc[idx]['Book-Title'],
            'Author': book.iloc[idx]['Book-Author'],
            'Publisher': book.iloc[idx]['Publisher'],
            'Similarity Score': round(score, 5)
        })

    result_df = pd.DataFrame(result)

    if return_similarities:
        return result_df, similarities
    else:
        return result_df


Memanggil fungsi rekomendasi yang telah dibuat

In [36]:
recommended_books = get_book_recommendations("The Mummies of Urumchi", top_n=5)
print(recommended_books)

         ISBN                    Title            Author  \
0  0393045218   The Mummies of Urumchi   E. J. W. Barber   
1  3791535714     Die SchildbÃ?Â¼rger.  Erich KÃ?Â¤stner   
2  033037401X  Midwinter of the Spirit      Phil Rickman   
3  0965813509      The throne of bones  Brian McNaughton   
4  0316734500  The Bookseller of Kabul    Asne Seierstad   

                    Publisher  Similarity Score  
0  W. W. Norton &amp; Company           1.00000  
1             Dressler Verlag           0.99991  
2              Pan Publishing           0.99990  
3             Terminal Fright           0.99990  
4               Little, Brown           0.99990  


Sistem rekomendasi berbasis content-based filtering berhasil menghasilkan daftar buku yang sangat relevan dengan buku inputan "The Mummies of Urumchi" karya E. J. W. Barber.
Rekomendasi ini didasarkan pada kesamaan fitur teks, yaitu kombinasi judul buku, nama penulis dan publisher, yang kemudian dianalisis menggunakan model Word2Vec.

##Model Development dengan Collaborative Filtering

Menyiapkan data

In [37]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(filtered_ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

Hyperparameter tuning (grid search)

In [38]:
param_grid = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)
gs.fit(data)

Memilih Model terbaik

In [39]:
print("Best RMSE:", gs.best_score['rmse'])
print("Best Params:", gs.best_params['rmse'])

Best RMSE: 3.4880839235887007
Best Params: {'n_factors': 100, 'lr_all': 0.005, 'reg_all': 0.1}


Train SVD dengan parameter dan RMSE terbaik

In [40]:
svd_best = gs.best_estimator['rmse']
svd_best.fit(trainset)

Fungsi rekomendaasi

In [41]:
def recommend_books_for_user(user_id, model, book_df, n_recommendations=5):
    all_isbn = book_df['ISBN'].unique()
    predictions = []
    for isbn in all_isbn:
        try:
            pred = model.predict(user_id, isbn)
            predictions.append((isbn, pred.est))
        except:
            continue

    top_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:n_recommendations]
    recommended_isbn = [isbn for isbn, _ in top_predictions]
    estimated_rating = [round(rating, 2) for _, rating in top_predictions]
    recommended_books = book_df[book_df['ISBN'].isin(recommended_isbn)][['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
    recommended_books = recommended_books.set_index('ISBN').loc[recommended_isbn].reset_index()
    recommended_books['Predicted-Rating'] = estimated_rating
    return recommended_books


Memanggil fungsi

In [42]:
sample_user_id = filtered_ratings['User-ID'].sample(1).values[0]
print(f"Sample user ID: {sample_user_id}")

recommendations = recommend_books_for_user(sample_user_id, svd_best, book, n_recommendations=5)
print(recommendations)

Sample user ID: 11676
         ISBN                                         Book-Title  \
0  193156146X                           The Time Traveler's Wife   
1  0553582143                                       Body of Lies   
2  0345413881       Dr. Death (Alex Delaware Novels (Paperback))   
3  0380720132  The Mystery of the Cupboard (Indian in the Cup...   
4  2253044903                Le Parfum : Histoire d'un meurtrier   

          Book-Author  Year-Of-Publication                Publisher  \
0  Audrey Niffenegger               2003.0  MacAdam/Cage Publishing   
1       IRIS JOHANSEN               2003.0                   Bantam   
2  Jonathan Kellerman               2001.0         Ballantine Books   
3    Lynne Reid Banks               1999.0             HarperTrophy   
4  Patrick SÃ?Â¼skind               1988.0                      LGF   

   Predicted-Rating  
0             10.00  
1             10.00  
2             10.00  
3              9.99  
4              9.95  
